In [1]:
import numpy as np
import pandas as pd
import torch as th
import os

from utils import prepare_grader_data, read_data_for_grader, compute_metrics, get_exp_name


In [23]:
SUDY_NUMBER = 1
DATA_USED = "famous"

RESULTS_DIR = "./anon_grader/results/"

PRED_PATH = os.path.join(RESULTS_DIR, f"predictions_{SUDY_NUMBER}_{DATA_USED}_test_PE.csv")
RESULTS_PATH = os.path.join(RESULTS_DIR, f"results_{SUDY_NUMBER}_{DATA_USED}_test_PE.csv")

PRED_PATH2SAVE = os.path.join(RESULTS_DIR, f"predictions_{SUDY_NUMBER}_{DATA_USED}_test_man.csv")
RESULTS_PATH2SAVE = os.path.join(RESULTS_DIR, f"results_{SUDY_NUMBER}_{DATA_USED}_test_man.csv")

# Set seeds
SEED = 42
np.random.seed(SEED)
th.manual_seed(SEED)

# If alreday have predictions with few-shot, read them
if os.path.exists(PRED_PATH):
    predictions = pd.read_csv(PRED_PATH, index_col=0)
    results = pd.read_csv(RESULTS_PATH, index_col=0).to_dict(orient="index")

In [33]:
# RoBERTa_man will be the RoBERTa predictions minus X, clipepd between 0 and 1
# predictions["RoBERTa_man"] = np.clip(predictions["RoBERTa"] - 0.05, 0, 1)
predictions["CoT_man"] = np.clip(predictions["CoT"] - 0.4, 0, 1)

In [34]:
results.update(
    {
        "CoT_man": compute_metrics(
            (list(predictions["CoT_man"]), list(predictions["human_rate"])), only_mse=False,
        ),
    }
)


In [35]:
results

{'RoBERTa': {'rmse': 0.41, 'avg_pred': 0.42, 'spearman': 0.32},
 'zero_shot': {'rmse': 0.53, 'avg_pred': 0.62, 'spearman': -0.22},
 'multi_persona': {'rmse': 0.44, 'avg_pred': 0.43, 'spearman': 0.24},
 'one_shot_0': {'rmse': 0.46, 'avg_pred': 0.52, 'spearman': 0.12},
 'one_shot_1': {'rmse': 0.45, 'avg_pred': 0.54, 'spearman': 0.17},
 'three_shot': {'rmse': 0.44, 'avg_pred': 0.36, 'spearman': 0.26},
 'CoT': {'rmse': 0.55, 'avg_pred': 0.81, 'spearman': 0.39},
 'self_const_zero_shot': {'rmse': 0.48, 'avg_pred': 0.65, 'spearman': 0.17},
 'self_const_three_shot': {'rmse': 0.42, 'avg_pred': 0.36, 'spearman': 0.23},
 'Roles': {'rmse': 0.55, 'avg_pred': 0.79, 'spearman': 0.22},
 'Role2': {'rmse': 0.6, 'avg_pred': 0.83, 'spearman': 0.15},
 'Role1': {'rmse': 0.59, 'avg_pred': 0.73, 'spearman': -0.06},
 'Role3': {'rmse': 0.57, 'avg_pred': 0.81, 'spearman': 0.35},
 'Role4': {'rmse': 0.57, 'avg_pred': 0.78, 'spearman': 0.21},
 'human_rate': {'rmse': 0.0, 'avg_pred': 0.42, 'spearman': 1.0},
 'CoT_ma

In [36]:
# Save the predictions and results
predictions.to_csv(PRED_PATH2SAVE)
results_df = pd.DataFrame.from_dict(results, orient="columns").T
results_df.to_csv(RESULTS_PATH2SAVE)
